# Identifying ACR species with Gurobi

In a chemical network equipped with Mass Action kinetics we define chemical species with absolute concentration robustness (ACR) as those whose concentration at steady state is independent of initial conditions. For a given chemical network with stoichiometric matrix $S$, reaction rate $v_j = K_j \prod_i{x_i^{s_{ij}}}$ and given reaction rate constant, $K$, ranges, we can identify ACR species through the following optimization problem:

\begin{align}
    \begin{aligned}
      \label{eq:4}
      &\log{x_{min_i}}, \log{x_{max_i}} = \min,\max_{\substack{ \log{x} \in \rm I\!R^m, \\
                            v,w,\log{K} \in \rm I\!R^n_{\geq 0}}} \; \log{x_i}
      \\
      &\mathrm{s.t.}
      \\
      &1.\;Sv = 0
      \\
      &2.\;w_j = \log{v_j}
      \\
      &3.\;w_j = \log{K_j} + \sum_i s_{ij} \log{x_i}
      \\
      &4.\;\log{x_{min}} \leq \log{x} \leq \log{x_{max}}
      \\
      &5.\;v_{min} \leq v \leq v_{max}
      \\
      &6.\;\log{K_{min}} \leq \log{K} \leq \log{K_{max}}
    \end{aligned}
\end{align}

The key constraint here is constraint 2, in which we relate $w_j = \log{v_j}$. This constraint is implemented in the <a href="www.gurobi.com">Gurobi solver</a> with a piece-wise linear approximation of the log function, and it works quite well for large scale chemical networks. Let's try this out in a genome scale metabolic model of _Escherichia coli_. We will find ACR species without any additional constraint and for any positive values of parameters $K$.

In [1]:
from gurobipy import GRB
from importlib import reload
import numpy as np
import pandas as pd
# from matplotlib import pyplot as plt
# import seaborn as sns
import parameters as par
import GEM_model
import gurobi_model

GEM = GEM_model.prepareGEM(path_to_GEM=f'{par.work_directory}/{par.model}',
                      carbon_source=par.carbon_source,
                      uptake_rate=par.uptake_rate,
                      loopless=False,
                      biomass_reaction_id=par.biomass_rxn_id)
GEM

Using license file C:\Users\tinta\gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file C:\Users\tinta\AppData\Local\Temp\tmpwu0t9xjb.lp
Reading time = 0.05 seconds
: 1147 rows, 4276 columns, 17168 nonzeros


Name,iML1515
Memory address,0x022b27004a48
Number of metabolites,1147
Number of reactions,2138
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## Some notes on results
Ok, so the idea was cool. Unfortunately, Gurobi is way to slow to solve this problem. Adding log constraints is fine I don't add the mass-action constraint $Sv=0$. However, requiring steady state plus $w = \log{v}$ is apparently too much to ask...

In [ ]:
reload(par)
reload(gurobi_model)
lp_model, variables = gurobi_model.buildLPModel(GEM)
logx_optimum = {}
for met in GEM.metabolites:
    print(met.id)
    logx_optimum[met.id] = {}
    
    lp_model.setObjective(variables['logx'][f'logx_{met.id}'], GRB.MINIMIZE)
    lp_model.update()
    lp_model.optimize()
    logx_optimum[met.id]['min'] = lp_model.objval
    
    
    lp_model.setObjective(variables['logx'][f'logx_{met.id}'], GRB.MAXIMIZE)
    lp_model.update()
    lp_model.optimize()
    logx_optimum[met.id]['max'] = lp_model.objval
    

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
dhap_c
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 3265 rows, 7561 columns and 16990 nonzeros
Model fingerprint: 0x7746b3f5
Model has 2138 general constraints
Variable types: 7561 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-06, 8e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Presolve added 5356 rows and 214051 columns
Presolve time: 2.61s
Presolved: 8621 rows, 221612 columns, 645760 nonzeros
Presolved model has 2089 SOS constraint(s)
Variable types: 221610 continuous, 2 integer (2 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    5109   -1.8418401e+01   3.417450e+04   0.000000e+00      5s
    7089   -1.8418044e+01   1.043956e+04   0.000000e+00     10s
    9129   -1.8417688e+01   3.769330e+03   0.000000e+00     15s
   11229   -1.8417458e+01  

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "C:\Users\tinta\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\iostream.py", line 385, in write
    if self.echo is not None:
KeyboardInterrupt


   253   293  -18.42068   79  377          -  -18.42068      -   136   91s
   358   359  -18.42068  107  364          -  -18.42068      -   136  105s
   432   448  -18.42068  128  378          -  -18.42068      -   147  126s
   535   519  -18.42068  151  378          -  -18.42068      -   155  153s
   622   561  -18.42068  181  377          -  -18.42068      -   175  172s
   672   651  -18.42068  194  387          -  -18.42068      -   177  194s
   783   652  -18.42068   64    0          -  -18.42068      -   185  196s
   785   653  -18.42068   94    0          -  -18.42068      -   185  220s
   816   651  -18.42068   29  343          -  -18.42068      -  17.3  225s
   836   670  -18.42068   34  352          -  -18.42068      -  22.0  230s
   848   681  -18.42068   37  362          -  -18.42068      -  22.5  235s
   877   720  -18.42068   41  348          -  -18.42068      -  30.0  246s
   919   769  -18.42068   51  352          -  -18.42068      -  34.1  260s
   990   808  -18.42068  